In [1]:
ls # start by reading in lyrics from the small dataset

Scrape tester.ipynb             poemfiles/
Study Data.ipynb                poetry_foundation_scraper.py
chromedriver*                   poetry_scraper_improved.py
full_scraper.py                 poetry_scraper_improved_old.py
glove/                          rhyme_dict_full.txt
lyrics_billboard.csv*           rhyme_dict_simple.txt
lyrics_large.csv*               rhyme_phones.txt
lyrics_small.csv*               scraper/
mlcourse/                       scraper.py
poem_VAE.py                     small_poem_set.csv


In [2]:
import pandas as pd
import csv
import re
import string

In [3]:
#data = pd.read_csv("lyrics_small.csv")

In [4]:
#data.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [5]:
#lyrics = data.loc[:,'text']

In [6]:
w = 'alpha-_\''
w2 = w.replace('-_\'', '')
print(w)
print(w2)
#print(w2.isalpha())

alpha-_'
alpha


In [7]:
def clean(w):
    return w.replace('-','').replace('—','').replace("\'",'')

def load_pronunciation_dict(simple=True):
    filename = "rhyme_dict_simple.txt" if simple else "rhyme_dict_full.txt"
    rhyme_dict = {}
    with open(filename, "r") as file:
        for line in file: # only add words, including characters ' - and —
            w = line.split()[0]
            if not w[0].isalpha() or not clean(w).isalpha() or w in rhyme_dict:
                continue
            rhyme_dict[w] = line.split()[1:]
    return rhyme_dict

def word_pronunciation(w, rhyme_dict):
    if w.upper() not in rhyme_dict:
        return w.upper()
    s = '\"'
    for c in rhyme_dict[w.upper()]:
        s += c + " "
    return s[:-1] + '\"'
            
def line_pronunciation(s, rhyme_dict): # returns pronunciation of line of words
    if len(s) == 0:
        return ""
    split = re.split('(\W+)', s) # remove punctuation !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ temporarily
    words = []
    if not (split[0].isalpha() and split[1] == "\'"):
        words.append(split[0].upper())
    for i in range(len(split) - 1):
        if split[i].isalpha() and split[i+1] == "\'":
            if i + 2 < len(split) and split[i+2].isalpha():
                words.append(split[i].upper() + "\'" + split[i+2].upper())
            else:
                words.append(split[i].upper() + "\'")
        else:
            words.append(split[i+1].upper())
    p = ""
    for w in words:
        p += word_pronunciation(w, rhyme_dict)
    return p

def pronunciation(s, rhyme_dict): # returns pronunciation of paragraph of words
    paragraph = s.split("\n")
    if len(paragraph) == 0:
        return ""
    p = ""
    for line in paragraph:
        p += line_pronunciation(line, rhyme_dict) + "\n"
    return p[:-1]

In [8]:
#rhyme_dict = load_pronunciation_dict(simple=False)
#print(rhyme_dict['HUNGRY'])

['HH', 'AH1', 'NG', 'G', 'R', 'IY0']


In [9]:
w = "hello"
l = "hello my name is John the blargskadarf"
p = "Autumn moonlight\na worm digs silently\ninto the chestnut"
#print(word_pronunciation(w, rhyme_dict))
#print(line_pronunciation(l, rhyme_dict))
#print(pronunciation(p, rhyme_dict))

"HH AH0 L OW1"
"HH AH0 L OW1" "M AY1" "N EY1 M" "IH1 Z" "JH AA1 N" "DH AH0" BLARGSKADARF
"AO1 T AH0 M" "M UW1 N L AY2 T"
"AH0" "W ER1 M" "D IH1 G Z" "S AY1 L AH0 N T L IY0"
"IH1 N T UW0" "DH AH0" "CH EH1 S N AH2 T"


In [10]:
#print(word_pronunciation("sensation", rhyme_dict))
#print(word_pronunciation("station", rhyme_dict))
#print(word_pronunciation("ovation", rhyme_dict))
#print(word_pronunciation("affiliation", rhyme_dict))
#print(word_pronunciation("translation", rhyme_dict))

"S EH0 N S EY1 SH AH0 N"
"S T EY1 SH AH0 N"
"OW0 V EY1 SH AH0 N"
"AH0 F IH2 L IY0 EY1 SH AH0 N"
"T R AE0 N Z L EY1 SH AH0 N"


In [11]:
#print(pronunciation(lyrics[0], rhyme_dict))

"L UH1 K" "AE1 T" "HH ER1" "F EY1 S", "IH1 T""IH1 T S""EH1 S" "AH0" "W AH1 N D ER0 F AH0 L" "F EY1 S"  
"AH0 N D" "IH1 T" "M IY1 N Z" "S AH1 M TH IH0 NG" "S P EH1 SH AH0 L" "T UW1" "M IY1"  
"L UH1 K" "AE1 T" "DH AH0" "W EY1" "DH AE1 T" "SH IY1" "S M AY1 L Z" "W EH1 N" "SH IY1" "S IY1 Z" "M IY1"  
"HH AW1" "L AH1 K IY0" "K AE1 N" "W AH1 N" "F EH1 L OW0" "B IY1"?  
  
"SH IY1 Z""EH1 S" "JH AH1 S T" "M AY1" "K AY1 N D" "AH1 V" "G ER1 L", "SH IY1" "M EY1 K S" "M IY1" "F IY1 L" "F AY1 N"  
"HH UW1" "K UH1 D" "EH1 V ER0" "B IH0 L IY1 V" "DH AE1 T" "SH IY1" "K UH1 D" "B IY1" "M AY1 N"?  
"SH IY1 Z""EH1 S" "JH AH1 S T" "M AY1" "K AY1 N D" "AH1 V" "G ER1 L", "W IH0 TH AW1 T" "HH ER1" "AY1""AY1 M""EH1 M" "B L UW1"  
"AH0 N D" "IH1 F" "SH IY1" "EH1 V ER0" "L IY1 V Z" "M IY1" "W AH1 T" "K UH1 D" "AY1" "D UW1", "W AH1 T" "K UH1 D" "AY1" "D UW1"?  
  
"AH0 N D" "W EH1 N" "W IY1" "G OW1" "F AO1 R" "AH0" "W AO1 K" "IH0 N" "DH AH0" "P AA1 R K"  
"AH0 N D" "SH IY1" "HH OW1 L D Z" "M IY1" "AH0 N D" "S K

In [12]:
import pronouncing
print(pronouncing.rhymes("cat"))

['arnatt', 'at', 'at-bat', 'balyeat', 'bat', 'batt', 'batte', 'begat', 'bhatt', 'biernat', 'blatt', 'brat', 'bratt', 'catt', 'chat', 'combat', 'dat', 'delatte', 'deslatte', 'elat', 'fat', 'flat', 'flatt', 'gat', 'gatt', 'glatt', 'gnat', 'gujarat', 'hat', 'hatt', 'hnat', 'inmarsat', 'jagt', 'kat', 'katt', 'klatt', 'krat', 'kratt', 'labatt', 'landsat', 'lat', 'mat', 'matt', 'matte', 'mcnatt', 'nat', 'non-fat', 'nonfat', 'pat', 'patt', 'platt', 'platte', 'pratt', 'pratte', 'prevatt', 'prevatte', 'rat', 'rat-a-tat', 'ratte', 'sadat', 'sarratt', 'sat', 'scat', 'schadt', 'shatt', 'slaght', 'slat', 'spat', 'splat', 'spratt', 'stat', 'tat', 'that', 'tit-for-tat', 'tvsat', 'vat']


In [17]:
# ls ./poemfiles # next we count the number of poems we've collected that have undefined words
poems = pd.read_csv("./poemfiles/poems.csv")

In [3]:
ls

GloVe Test.ipynb                poem_VAE.py
Image VAE.ipynb                 poemfiles/
Poem LSTM.ipynb                 poetry_foundation_scraper.py
Poem_LSTM.py                    poetry_scraper_improved.py
Study and Clean Data.ipynb      poetry_scraper_improved_old.py
chromedriver*                   results/
edited_poems.csv                rhyme_dict_full.txt
full_scraper.py                 rhyme_dict_simple.txt
glove/                          rhyme_phones.txt
lyrics_billboard.csv*           scraper/
lyrics_large.csv*               scraper.py
lyrics_small.csv*               small_poem_set.csv
mlcourse/


In [75]:
import pandas as pd
import re
data = pd.read_csv('./poemfiles/poems.csv')
temp = data.iloc[:,3].apply(str)
newdata = data[temp != 'nan']
newpoemdata = newdata.iloc[:,3].apply(lambda s: s.replace('\xa0', ' '))
print(len(newpoemdata))

15652


In [94]:
import string
string.punctuation
valid = set()
for c in string.punctuation: valid.add(c)
for c in ['‘', '’', '⎯', '—', '─', '–']: valid.add(c)
print(valid)

def check_valid(s):
    for c in s:
        if c.isalnum() or c in valid or c.isspace():
            pass
        else:
            return False
    return True

{'_', '⎯', '$', '}', '—', '~', '>', '(', '?', '[', "'", '{', ',', '─', '#', '"', '+', '–', '*', '!', '^', ')', '.', '\\', '@', ':', '`', '|', '/', '=', '‘', '’', '<', ']', '%', '-', ';', '&'}


In [107]:
changes = []
invalid = set()
correct = []
for poem in newpoemdata:
    if not check_valid(poem):
        newpoem = ""
        for c in poem:
            if c.isalnum() or c in valid or c.isspace(): newpoem += c
            else: invalid.add(c)
        changes.append(poem)
    else:
        correct.append(poem)

print(len(changes))

2774


In [97]:
print(invalid)

{'»', '̄', '§', '̈', '≈', '❖', '́', '\u200b', '™', '→', '̧', '®', '‚', '▶', '♥', '€', '•', '❏', '◀', '̃', '×', '”', '¤', '☽', '£', '¡', '·', '˚', '°', '«', '¢', '¿', '★', '\xad', '�', '̀', '♂', '\x9f', '\ufeff', '“', '…', '´'}


In [116]:
mapping = [('‘', '\''), ('’', '\''), ('⎯', '-'), ('—', '-'), ('─', '-'), ('–', '-')]
new_correct = []
for poem in correct:  
    for k, v in mapping:
        poem = poem.replace(k, v)
    new_correct.append(poem)
    
for poem in new_correct:
    for c in poem:
        if c in ['‘', '’', '⎯', '—', '─', '–']:
            print(c)

In [109]:
final_poems = pd.DataFrame({'poems':new_correct})

In [118]:
final_poems.head()

,poems
0,"Philosophic\nin its complex, ovoid emptiness,\..."
1,We'd like to talk with you about fear t...
2,The Wise Men will unlearn your name.\nAbove yo...
3,WINTER\nMore time is spent at the window.\n\nS...
4,May I never be afraid\nespecially of myself\nb...


In [111]:
'\'' in ['‘', '’', '⎯', '—', '─', '–']

False

In [112]:
'-' in ['‘', '’', '⎯', '—', '─', '–']

False

In [120]:
final_poems.to_csv("clean_poems.csv")

In [1]:
ls

GloVe Test.ipynb                mlcourse/
Image VAE.ipynb                 poem_VAE.py
Poem LSTM.ipynb                 poemfiles/
Poem_LSTM.py                    poetry_foundation_scraper.py
Study and Clean Data.ipynb      poetry_scraper_improved.py
chromedriver*                   poetry_scraper_improved_old.py
clean_poems.csv                 results/
edited_poems.csv                rhyme_dict_full.txt
full_scraper.py                 rhyme_dict_simple.txt
glove/                          rhyme_phones.txt
lyrics_billboard.csv*           scraper/
lyrics_large.csv*               scraper.py
lyrics_small.csv*               small_poem_set.csv


In [122]:
edited = pd.read_csv("edited_poems.csv")

In [124]:
edited.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15652 entries, 0 to 15651
Data columns (total 4 columns):
Author                  15652 non-null object
Title                   15652 non-null object
Poetry Foundation ID    15652 non-null int64
Content                 15652 non-null object
dtypes: int64(1), object(3)
memory usage: 489.2+ KB


In [4]:
import pandas as pd
data = pd.read_csv("clean_poems.csv")

In [6]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [8]:
def clean_poem(poem):
    temp = poem.replace('‘', '\'')
    temp = temp.replace('’', '\'')
    temp = temp.replace('⎯', '-')
    temp = temp.replace('—', '-')
    temp = temp.replace('─', '-')
    temp = temp.replace('–', '-')
    return temp

s = '‘’⎯—─–'
print(s)
print(clean_poem(s))

‘’⎯—─–
''----


In [9]:
series = data.iloc[:,1]
clean_series = series.apply(clean_poem)

for poem in clean_series:
    for c in poem:
        if c in ['‘', '’', '⎯', '—', '─', '–']:
            print(c)

In [13]:
clean_series.to_csv("../Sentence-VAE/data/ultimate_poems.csv")